In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from PIL import Image
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2 as cv
import torchlayers as tl
import os

In [ ]:
trans = transforms.Compose([transforms.Resize((120, 120)), transforms.ToTensor()])

# datas = torchvision.datasets.ImageFolder(root = './data/kcar/lights', transform = trans)
# train_size = int(len(datas) * 0.8)
# test_size = len(datas) - train_size
# train_sets, test_sets = torch.utils.data.random_split(datas, [train_size, test_size])

# edge_datas = torchvision.datasets.ImageFolder(root = './data/kcar_edge', transform = trans)
# edge_train_size = int(len(edge_datas) * 0.8)
# edge_test_size = len(edge_datas) - edge_train_size
# edge_train_sets, edge_test_sets = torch.utils.data.random_split(edge_datas, [edge_train_size, edge_test_size])

edge_train_sets = torchvision.datasets.ImageFolder(root = './data/kcar_edge/lights', transform = trans)
edge_test_sets = torchvision.datasets.ImageFolder(root = './data/kcar_edge/darks', transform = trans)
train_sets = torchvision.datasets.ImageFolder(root = './data/kcar/lights', transform = trans)
test_sets = torchvision.datasets.ImageFolder(root = './data/kcar/darks', transform = trans)
labels = train_sets.classes
print(edge_train_sets)
print(train_sets)
# print(test_sets)
# print(edge_test_sets)

# print(len(labels))
# for i in range(4):
#     img, label = train_sets[i]
#     print(labels[label])
#     plt.subplot(241 + i)
#     plt.imshow(np.clip(img.permute(1,2,0),0,1))
#     plt.tight_layout()
#     plt.show()
    # save_image(img, str(i)+'_.jpg')


In [ ]:
train_loader = DataLoader(train_sets, batch_size = 32, num_workers = 2)
edge_train_loader = DataLoader(edge_train_sets, batch_size = 32, num_workers = 2)
test_loader = DataLoader(test_sets, batch_size = 32)
edge_test_loader = DataLoader(edge_test_sets, batch_size = 32)

In [ ]:
model1 = torch.nn.Sequential(
    tl.Conv(64),  # specify ONLY out_channels
    tl.ReLU(),  # use torch.nn wherever you wish
    tl.BatchNorm(),  # BatchNormNd inferred from input
    tl.Conv(128),  # Default kernel_size equal to 3
    tl.ReLU(),
    tl.MaxPool(),
    tl.Conv(256, kernel_size=11),  # "same" padding as default
    # tl.ReLU(),
    tl.GlobalMaxPool(),
    tl.Linear(52),  # Output for 52 classes
)
model1 = tl.build(model1, torch.randn(1, 3, 120, 120))
model1.cuda()
model2 = torch.nn.Sequential(
    tl.Conv(64),  # specify ONLY out_channels
    torch.nn.ReLU(),  # use torch.nn wherever you wish
    tl.BatchNorm(),  # BatchNormNd inferred from input
    tl.Conv(128),  # Default kernel_size equal to 3
    tl.ReLU(),
    tl.MaxPool(),
    tl.Conv(256, kernel_size=11),  # "same" padding as default
    # tl.ReLU(),
    tl.GlobalMaxPool(),
    tl.Linear(52),  # Output for 52 classes
)
model2 = tl.build(model2, torch.randn(1, 3, 28, 28))
model2.cuda()

# one_way_model = torch.nn.Sequential(
#     tl.Conv(64),  # specify ONLY out_channels
#     tl.ReLU(),  # use torch.nn wherever you wish
#     tl.BatchNorm(),  # BatchNormNd inferred from input
#     tl.Conv(128),  # Default kernel_size equal to 3
#     tl.ReLU(),
#     tl.Conv(256, kernel_size=11),  # "same" padding as default
#     tl.GlobalMaxPool(),  # Known from Keras
#     tl.Linear(52),  # Output for 2 classes
# )
# one_way_model = tl.build(one_way_model, torch.randn(1, 3, 28, 28))
# one_way_model.cuda()

In [ ]:
class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.fc = nn.Linear(104, 52)
        
    def forward(self, x1, x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        x = torch.cat((x1, x2), dim=1)
        # print(x1.shape, x.shape, self.num_flat_features(x))
        x = x.view(-1, self.num_flat_features(x))
        # print(x.shape)
        # print(x)
        x = self.fc(x)
        # print(x.shape)
        # print(x)
        return x

    def num_flat_features(self, x):  
        size = x.size()[1:] 
        num_features = 1 
        for s in size: 
            num_features *= s 
        return num_features

model = MyEnsemble(model1, model2)
model.cuda()

In [ ]:
import torch.optim as optim
criterion = torch.nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2,5,8], gamma=0.1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')
print(torch.cuda.is_available())
torch.cuda.empty_cache()

In [ ]:
model.train()
print(len(train_loader))
for epoch in range(10):
    for index, ((data, target), (edge_data, edge_target)) in enumerate(zip(train_loader, edge_train_loader)):
        data, target = data.cuda(), target.cuda()
        edge_data, edge_target = edge_data.cuda(), edge_target.cuda()

        output = model(data, edge_data)
        optimizer.zero_grad()
        print(target)
        print(output)
        loss = criterion(output, target)
        print(loss.item())
        loss.backward()
        optimizer.step()

        if index % 10 == 0:
            print("loss of {} epoch, {} index : {}".format(epoch, index, loss.item()))
    
    scheduler.step()
    print("-----")


In [ ]:
model.train()  # 학습
print(len(train_loader))
for epoch in range(10): 
    optimizer.zero_grad()  # 기울기 초기화
    for index, ((data, target), (edge_data, edge_target)) in enumerate(zip(train_loader, edge_train_loader)):
        data, target = data.cuda(), target.cuda()
        edge_data, edge_target = edge_data.cuda(), edge_target.cuda()

        output = model(data, edge_data)
        loss = criterion(output, target)
        # print(loss.item())
        loss.backward()  # 역전파
        optimizer.step()

        if index % 10 == 0:
            print("loss of {} epoch, {} index : {}".format(epoch, index, loss.item()))
    scheduler.step()    
    print("scheduler-step")
    print("-----") 